# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

It is a classification problem.

As we are expecting a binary result here which should be "need early intervention" and "does not need early intervention". That's why we need to use classification model to classify which type of students need to go to intervention from the given data. 

If we are going to do examination score prediction, I would adopt regression model as it suitable for predicting continous numeric result.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) -1

# TODO: Calculate passing students
n_passed = student_data['passed'].value_counts()['yes']

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = (float(n_passed) / n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = train_test_split(X_all, y_all, test_size = 95, random_state = 2)

# Set the number of testing points
num_test = X_all.shape[0] - num_train[0]

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = num_train[0]
X_test = num_train[1]
y_train = num_train[2]
y_test = num_train[3]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

I would adopt Support Vector Machine (SVM), Decision Tree Classifer, and Naive Bayes Guassian method to be the supervised learning models. 

For SVM, it suits for doing data classification to find the best hyperplane between two types of data that given different output in a binary situation such as true or flase, etc. And SVM can efficiently perform a non-linear classification using what is called the kernel trick, implicitly mapping their inputs into high-dimensional feature spaces. 

- Strengths:First, it has a regularisation parameter, which could help the user to avoid overfitting. Second, we can apply Kernel Trick as it can help us to do fast and efficient computation on large dimensional space which means we can handle XOR where Decision cannot. Lastly, we could adopt Sequential Minimal Optimization Algorithm to fasten its computation speed. Uses a subset of training points in the decision function that called support vectors, so it is also memory efficient.


- Weaknesses:  If the number of features is much greater than the number of samples, the method is likely to give poor performances. And SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation Moreover kernel models can be quite sensitive to over-fitting the model selection criterion.

SVM application: It is widely applied in biological and science field, such as classify proteins with up to 90% of the compounds classified correctly. 


For Decision Tree Classifier, it is a data classification model and it uses layers of filter to determine value of output and classify a binary result. The flow is like, we pick the best attributes, then we asked it a question, it would output a new attribute, we ask it another question and repeat this step to dig deep to the final answer. 

- Strengths: Each layer would exclude approximately half of the data out, so it could help us to narrow down to our result so effective. We could use Top-Down Learning algorithm (ID3) to help us calculate the entropy of every attribute using the data set. The reason why entropy is extremely useful is because it is a measure of impurity in a bunch of examples that control how a decision tree decides where to split the data. Then, it is simple to understand and to interpret as the trees can be visualised. Next, it can handle both numerical and categorical data which give us larger flexibility to choose the features as some model cannot handle categorical data.

- Weakness: Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree. Moreover, for the decision tree expressiveness, we need to avoid using XOR as it could increase the computation load incredibly. And lastly, Decision Tree learners can create over-complex trees that do not generalise the data well which refers to overfitting. Mechanisms such as setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.

Decision Tree Classifier application: It applys in library book use prediction and look for the characterization of Leiomyomatous Tumors, etc. 

For Naives Bayes Guassian Method, it is also a data classification model. It simply calculate the possibility of an outcome according to the Bayes Rule and the prior we set. And we would use the posteriors to do prediction.

- Strengths: Compare to other more sophisticated models, it is an extremely fast method. And according to the prior and the Bayes Rule, we can calculate the estimate probability of an outcome occurence instead of approximate it by putting a curve among a dataset. Also the value is interpretable such as 95%. And lastly, it builds a convenient setting for a wide range of models, such as hierarchical models and missing data problems. 

- Weakness: It is known to be a bad estimator, so the probability outputs from predict_proba are not to be taken too seriously.

Naives Bayes Guassian Application: It applies on checking the email is spam or not, check a a piece of text expressing positive emotions, or negative emotions, etc.


And the reasons I choose this three models are because:

1. We are not doing regression prediction as our aim is to determine the student is going to pass or not according to the given data. Therefore, classification model is the appropriate way to classify whose going to pass or not. However, if we want to predict the student's examination score, I would adopt regression model instead.

2. These three models are relatively simple. SVM aims to use a linear curve to split the student who gonna pass or fail the exam. Simply split it half. Decision Tree simply decide the student would pass by evalutaing who they are and what they did. And Bayes Learning model is simply calculating every possibility that lead to pass and failure.

3. From the data I achieved, there are less than 400 entities which means SVM can fully utilise here withput concerning the speed or too much data problem. And there is not only numerical data but categotized data too, therefore, we need to choose model that could handle these two types of data.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [9]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# TODO: Initialize the three models
clf_A = SVC(random_state=0)
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = GaussianNB()

# TODO: Set up the training set sizes
# I combined this part with the 'train_predict' implementation part.

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
        print '\n'


Training a SVC using a training set size of 100. . .
Trained model in 0.0047 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8905.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.8392.
Training a SVC using a training set size of 200. . .
Trained model in 0.0032 seconds
Made predictions in 0.0023 seconds.
F1 score for training set: 0.8620.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.8267.
Training a SVC using a training set size of 300. . .
Trained model in 0.0066 seconds
Made predictions in 0.0044 seconds.
F1 score for training set: 0.8504.
Made predictions in 0.0015 seconds.
F1 score for test set: 0.8267.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0035 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6875.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Machine (SVM) **  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0011s            |            0.0007s            |          0.8905         |          0.8392        |
| 200               |           0.0023s            |            0.0011s            |          0.8620         |          0.8267        |
| 300               |           0.0044s            |            0.0015s            |          0.8504         |          0.8267        |

** Classifer 2 -  Decision Tree Classifier **  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0012s            |           0.0003s             |              1             |          0.6875        |
| 200               |            0.0003s            |           0.0002s             |              1             |          0.6557        |
| 300               |            0.0002s            |           0.0002s             |              1             |          0.7883        |

** Classifer 3 - Naives Bayes Guassian **  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0003s             |           0.0003s             |         0.4103         |         0.2955        |
| 200               |            0.0004s             |           0.0002s             |         0.3544         |         0.2651        |
| 300               |            0.0060s             |           0.0018s             |         0.8504         |         0.8267        |


## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Generally I would choose Support Vector Machine to be my supervised model as it could calculate the highest score and from each training data size, it still can give the most consistent result. Especially, it can predict the best result by using the least amount of training data. Although its run time is the slowest one for each dataset size. However, when we look at the fastest one, the decision tree model, it can't even reach 0.8 with its best score that trained with 300 data. Lastly, the Naive Bayes Guassian Model require 300 data to give the second best result as same as SVM that trained with 200 and 300 data. 

Consider if we do not have 300 data but 100 only. SVM can predict an exceptional result among these models. You might say what if we have limited time to run the model, I would say it is pointless to run an in accurate model no matter how fast it is as it provides useless result. Moreover, the svm train time for 300 data is 0.0047s. Although it is the slowest one among these models, however, human would not feel anything about this. And assume there are 3,0000 student in this local school, the run time is 0.047s only. We would completely satisfy with this prediction speed. Time would not be the primority parameter we would concern here. Accuracy and consistency with different data size are much more important factors we have to concern in this model.

As a result SVM is the best model suggested from this experiment.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

The model I have chosen is Support Vector Machine (SVM).

The prediction mechanism is simple. At the beginning, I would like to clarify our objective of using SVM which aims to use a boundary to split the students into two groups. 

Let's imagine there is a 1m x 1m square, there are two groups of tennis balls blue and red placed in it which represent student passed exam and failed exam respectively. Each of their position is determined by their own unique features that refers to the same color tennis ball could locate at different coordination of the square. What we want to do is putting a wood plane inside the square with the width as largest as possible to seperate two groups of ball. With the largest width of the wood plane, we could maximize the distance between two groups of ball to achieve the most objective and accurate result.
The reason to aim for the largest width is to achieve the most significant and objective boundary falls between two groups of balls.

However, two groups of balls might not be able to split with a straight line which is the ideal result. Therefore we would approximate the result by adopting similarity measure to determine the balls are close or far.

After we put the wood plane inside the square, we can start our preodiction. We only need to evaluate all the student features and determine their position. Afterthat we could find out it falls at pass or fail area to get the prediction result that the student need to have early intervention or not.




### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [18]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':['linear'],'gamma':[10]}

# TODO: Initialize the classifier
clf = SVC(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0026 seconds.
Tuned model has a training F1 score of 0.8380.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8116.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F1 score for training is 0.8380 and testing is 0.8116. Compare to the untuned model, both of its training and testing score are lower than the untuned one. Howeer, the training and testing scores are much closer.Which refers to a better prediction accuracy.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.